In [15]:
from dotenv import load_dotenv
load_dotenv()
# import os
# print(os.getenv('REQUESTS_CA_BUNDLE'))

True

## Accessing OpenAI LLM model using Langchain

In [6]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.6)

Temperature (ranges b/w 0 and 1) : how creative we want our model to be.  

0 -> it means model is very safe it is not taking any bets. (The more closer it is towards 0, the outputs will tend to be more or less the same for same prompt)  

1 -> It will take risk it might generate wrong output but it is very creative. (The more closer it is towards 1, the more creative liberty it takes which may give diverse responses if we run same prompt multiple times)

In [16]:
text = "What is the capital of China"
print(llm.predict(text))



The capital of China is Beijing.


## Accessing Huggingface Open-Source LLM model using Langchain

In [18]:
from langchain import HuggingFaceHub
llm_hf = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    model_kwargs={"temperature":0, "max_length":64}
)

/Users/naushads/Documents/jupyter_notebooks/langchain-demo/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/naushads/Documents/jupyter_notebooks/langchain-demo/venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [19]:
print(llm_hf.predict("Can you tell me the capital of Russia?"))

moscow


## Prompt Templates and Langchain LLMChain

In [20]:
from langchain.prompts import PromptTemplate

country_capital_prompt_template = PromptTemplate(
    input_variables=["country"],
    template="Can you tell me the capital of {country}"
)
country_capital_prompt_template.format(country="India")


'Can you tell me the capital of India'

In [21]:
from langchain.chains import LLMChain

llm_country_capital_chain = LLMChain(
    llm=llm, 
    prompt=country_capital_prompt_template
)
print(llm_country_capital_chain.run("India"))



The capital of India is New Delhi.


In [22]:
llm_hf_country_capital_chain = LLMChain(
    llm=llm_hf, 
    prompt=country_capital_prompt_template
)
print(llm_hf_country_capital_chain.run("India"))

chennai


## Combining Multiple Chains Using Simple Sequential Chain

In [23]:
country_capital_prompt_template = PromptTemplate(
    input_variables=["country"],
    template="Can you tell me the capital of {country}"
)

llm_country_capital_chain = LLMChain(
    llm=llm, 
    prompt=country_capital_prompt_template
)

famous_places_in_capital_city_prompt_template = PromptTemplate(
    input_variables=["capital"],
    template="Suggest me some amazing places to visit in {capital}"
)

llm_famous_places_in_capital_city_chain = LLMChain(
    llm=llm, 
    prompt=famous_places_in_capital_city_prompt_template
)

In [24]:
from langchain.chains import SimpleSequentialChain
famous_places_in_capital_city_chain = SimpleSequentialChain(
    chains=[
        llm_country_capital_chain, 
        llm_famous_places_in_capital_city_chain
    ]
)
print(famous_places_in_capital_city_chain.run("India"))

 This city is a great mix of modern and ancient India. With its many monuments, temples, and galleries, it is a must-see for any traveler. Other places to visit in India include:

1. Taj Mahal, Agra: One of the world's most iconic monuments, the Taj Mahal is a symbol of love and beauty.

2. Jaipur: The "Pink City" of Rajasthan, Jaipur is a vibrant city known for its forts, palaces, and bazaars.

3. Goa: A popular beach destination, Goa has stunning beaches and a lively nightlife.

4. Kerala: Nicknamed "God's Own Country," Kerala is known for its lush green landscapes, traditional villages, and backwaters.

5. Varanasi: One of the holiest cities in India, Varanasi is a must-visit for its ancient temples, ghats, and spiritual atmosphere.

6. Leh-Ladakh: Located in the Himalayas, this mountain region is known for its rugged beauty, monasteries, and adventure activities.

7. Andaman & Nicobar Islands: These tropical islands offer stunning beaches, crystal clear


## Sequential Chain

In [25]:
country_capital_prompt_template = PromptTemplate(
    input_variables=["country"],
    template="Can you tell me the capital of {country}"
)

llm_country_capital_chain = LLMChain(
    llm=llm, 
    prompt=country_capital_prompt_template,
    output_key="capital"
)

famous_places_in_capital_city_prompt_template = PromptTemplate(
    input_variables=["capital"],
    template="Suggest me some amazing places to visit in {capital}"
)

llm_famous_places_in_capital_city_chain = LLMChain(
    llm=llm, 
    prompt=famous_places_in_capital_city_prompt_template,
    output_key="places"
)

In [26]:
from langchain.chains import SequentialChain
chain = SequentialChain(
    chains=[
        llm_country_capital_chain,
        llm_famous_places_in_capital_city_chain
    ],
    input_variables=[
        "country"
    ],
    output_variables=[
        "capital",
        "places"
    ]
)

In [28]:

print(
    chain(
        {
            "country":"India"
        }
    )
)

{'country': 'India', 'capital': '\n\nThe capital of India is New Delhi.', 'places': " Here are some amazing places to visit in New Delhi: \n\n1. Red Fort: This iconic fort in the heart of the city is a must-see. It was built in the 17th century and is a great example of Mughal architecture. \n\n2. India Gate: This imposing structure is a memorial to the martyrs of World War I and a popular tourist spot. \n\n3. Jama Masjid: One of the largest mosques in India, Jama Masjid is a beautiful sight. It was built in the 17th century by Mughal Emperor Shah Jahan. \n\n4. Humayun's Tomb: This is a UNESCO World Heritage Site and a great example of Mughal architecture. It was built in the 16th century and is the tomb of the Mughal Emperor Humayun. \n\n5. Qutub Minar: This is one of the oldest and tallest minarets in India. It was built in the 13th century and is a great example of Indo-Islamic architecture. \n\n6. Lotus Temple: This is a stunning architectural marvel and a popular tourist destinati

## Chat models using ChatOpenAI

In [29]:
from langchain.chat_models import ChatOpenAI
chatllm = ChatOpenAI(
    temperature=0.6,
    model="gpt-3.5-turbo"
)

In [30]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
chatllm([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Please provide some comedy punchlines on AI")
])

AIMessage(content='1. "Artificial intelligence is great, until it starts sending you passive-aggressive error messages like \'Did you really think that was a good idea?\'"\n2. "They say AI is the future, but I\'m still waiting for a robot chef that can actually make a decent sandwich."\n3. "AI can do a lot of things, but can it make your ex\'s new relationship status magically disappear from your Facebook feed? Asking for a friend."\n4. "I asked Siri if she believes in love at first sight. She responded, \'I\'m sorry, I can\'t answer that. But I can search the web for you.\' Thanks for nothing, Siri."\n5. "They say AI is getting smarter every day, but I still can\'t get my virtual assistant to understand my mom\'s voicemail instructions."\n6. "I tried talking to my smart speaker, but it just started playing \'Baby Shark\' on repeat. I think it\'s trying to drive me insane."\n7. "I asked Alexa if she had any dating advice for me. She said, \'Just be yourself.\' Thanks, Alexa, I guess ev

## Prompt Template + LLM + Output Parsers

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [32]:
chatllm = ChatOpenAI(
    temperature=0.6,
    model="gpt-3.5-turbo"
)

In [33]:
class CommaSeperatedOutput(BaseOutputParser):
    def parse(self, text):
        return text.strip().split(",")

In [34]:
chatmodel_system_message_template = "You are a helpful assistant. \
    When the user gives any input word, you should generate 5 synonyms of the input word \
    in a comma seperated manner."
chatmodel_human_message_template = "{text}"

chatprompt = ChatPromptTemplate.from_messages({
    ("system", chatmodel_system_message_template),
    ("human", chatmodel_human_message_template)
})

In [35]:
chain = chatprompt | chatllm | CommaSeperatedOutput()
chain.invoke({"text":"intelligent"})

['Sure! Please provide a word for which you would like to generate synonyms.']